In [9]:
# import pandas as pd
# from tqdm import tqdm
# import warnings

# warnings.filterwarnings("ignore")

# # A function which convert the column of "likely_benign", "likely_pathogenic" and "ambiguous" in to labels of 0, 1 and 2
# def convert_label(x):
#     assert x in ["likely_benign", "likely_pathogenic", "ambiguous"]
#     if x == "likely_benign":
#         return 0
#     elif x == "likely_pathogenic":
#         return 1
#     elif x == "ambiguous":
#         return 2
#     return pd.np.nan

# # load Alphamissence data from tsv to data frame
# aa_substitution = pd.read_csv('/home/jiahuih/data/jiahuih/Ucloud/Documents/SLC/Resolution/PMM/Alphamissense/AlphaMissense_hg38.tsv',sep='\t', skiprows=3, chunksize=100000)
# PMM_data = pd.read_csv('/home/jiahuih/data/jiahuih/Ucloud/Documents/SLC/Resolution/PMM/data/SLC6_UniprotClinvarLitVar_Pathogenicity.csv')
# output = pd.DataFrame()
# for chunk in tqdm(aa_substitution):
#     columns = ['transcript_id', 'protein_variant', "am_class", "am_pathogenicity"]
#     details = chunk[columns]
#     details['count'] = 1
#     details['am_class'] = details['am_class'].apply(convert_label)
#     groupup_allele = details.groupby(columns).sum().reset_index()
#     output = output.append(groupup_allele, ignore_index=True)

# final_output = output.groupby(columns).sum().reset_index()
# final_output.to_csv('/home/jiahuih/data/jiahuih/Ucloud/Documents/SLC/Resolution/PMM/Alphamissense/aggregated_hg38.csv', index=False)

717it [11:11,  1.07it/s]


In [1]:
import pandas as pd
PMM_data = pd.read_csv('../data/SLC6_UniprotClinvarLitVar_Pathogenicity_fullTranscript.csv')

aggregated_hg38 = pd.read_csv('aggregated_hg38.csv')

In [29]:
SLC6A8 = pd.read_csv('../data/SLC6A8_test_Resolution.csv')

In [8]:
# Split the mutation of DeepMind data into three columns: original, position and mutation
import re

def split_variant(x):
    return re.match(r"([A-Z])([0-9]+)([A-Z])",x).groups()

aggregated_hg38["original"], aggregated_hg38["position"], aggregated_hg38["mutat"] = zip(*aggregated_hg38["protein_variant"].apply(split_variant))

# ignore the transcript content after comma
aggregated_hg38['transcript_id'] = aggregated_hg38['transcript_id'].apply(lambda x: x.split(".")[0])

In [23]:
# join aggregated_hg38 onto PMM_data by the coloumns

aggregated_hg38['transcript_id'] = aggregated_hg38['transcript_id'].astype(str)
PMM_data['transcript_id'] = PMM_data['transcript_id'].astype(str)
aggregated_hg38['position'] = aggregated_hg38['position'].astype(int)
PMM_data['Position'] = PMM_data['Position'].astype(int)
aggregated_hg38['mutat'] = aggregated_hg38['mutat'].astype(str)
PMM_data['MutanAA'] = PMM_data['MutanAA'].astype(str)
aggregated_hg38['original'] = aggregated_hg38['original'].astype(str)
PMM_data['OriginalAA'] = PMM_data['OriginalAA'].astype(str)

merged_df = pd.merge(
    aggregated_hg38,
    PMM_data,
    how='right', 
    left_on=['transcript_id', 'position', 'mutat', 'original'],
    right_on=['transcript_id', 'Position', 'MutanAA', 'OriginalAA']
)

In [32]:
# join aggregated_hg38 onto SLC6A8 by the coloumns

aggregated_hg38['transcript_id'] = aggregated_hg38['transcript_id'].astype(str)
SLC6A8['transcript_id'] = SLC6A8['transcript_id'].astype(str)
aggregated_hg38['position'] = aggregated_hg38['position'].astype(int)
SLC6A8['Position'] = SLC6A8['Position'].astype(int)
aggregated_hg38['mutat'] = aggregated_hg38['mutat'].astype(str)
SLC6A8['MutanAA'] = SLC6A8['MutanAA'].astype(str)
aggregated_hg38['original'] = aggregated_hg38['original'].astype(str)
SLC6A8['OriginalAA'] = SLC6A8['OriginalAA'].astype(str)

merged_df_SLC6A8 = pd.merge(
    aggregated_hg38,
    SLC6A8,
    how='right', 
    left_on=['transcript_id', 'position', 'mutat', 'original'],
    right_on=['transcript_id', 'Position', 'MutanAA', 'OriginalAA']
)


In [26]:
print("all : ")
print(merged_df.shape)
print("="*30)
print("founded : ")
print(merged_df.dropna().shape)

all : 
(256, 16)
founded : 
(126, 16)


In [34]:
merged_df_SLC6A8.to_csv('../data/SLC6A8_test_am.csv', index=False)